# L20-0921 Aisha Muhammad Nawaz
## BSCS 7A  Data Science  
## Fall 2023 Assignment # 1 Web Scrapping
### Website
#### For this assignment you have to collect the data of all faculty members from the following website.
##### http://lhr.nu.edu.pk/faculty/
##### http://isb.nu.edu.pk/Faculty/allfaculty/
##### https://khi.nu.edu.pk/


In [1]:
urls=['https://lhr.nu.edu.pk/faculty/','http://isb.nu.edu.pk/Faculty/allfaculty/','https://khi.nu.edu.pk/']

## Step 1:
### You have to collect the following details of the faculty members for each department and for each campus.
### -> ID (Integer)
### -> Name  (String)
### -> Designation (String)
### -> HEC Approved PHD Supervisor (Bool)	
### -> Highest Education (String)
### -> Email	(Email)
### -> Department	(String)
### -> Extension (Integer)
### -> ImageURL (URL)

### Collect the data of the Lahore campus faculty members in the above format and transform it into a data frame. Save the data frame as a .csv file named lhr.csv. Do the same for Islamabad, Karachi and save the files as isb.csv and  khi.csv respectively. 

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
print('All Required Libraries Imported Successfully!')

All Required Libraries Imported Successfully!



# ~ LAHORE CAMPUS ~

In [3]:
requestLHR=requests.get(urls[0])
if(requestLHR.status_code==200):
    print(requestLHR,'\nHTTP GET Request to Lahore Campus Website Web Server Successful!\n')
    soupLHR=BeautifulSoup(requestLHR.content,'html.parser') #Parsing Data Retrieved
    print(soupLHR.prettify()[0:100])
else:
    print(requestLHR,'\nHTTP GET Request to Lahore Campus Website Web Server Unsuccessful!\n')

<Response [200]> 
HTTP GET Request to Lahore Campus Website Web Server Successful!

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-wid


In [4]:
# First Writing Code to scrap details of a single Faculty Member of Lahore Campus
LHRSinglefacultyMember=soupLHR.find('div',class_='facultyCard')
print(LHRSinglefacultyMember)

<div class="card facultyCard card-br-0 ml-0 mr-0 pb-0">
<div class="facultyImage">
<img class="card-img-top rounded-circle mt-3 mb-0 d-block mx-auto" src="/media/Faculty/02_-_Dr._Kashif_Zafar_Prof__HoD_CS.png"/>
</div>
<div class="card-body pr-0 pl-0 pb-0">
<h5 class="text-center">Dr. Kashif Zafar</h5>
<p class="small text-center font-italic">Professor &amp; HOD
                        
                         <br> HEC Approved PhD Supervisor 
                        
                        </br></p>
<p class="mb-0 text-center">kashif.zafar@nu.edu.pk</p>
<!--  <p class="mt-0 small text-center">ext no: 221</p> -->
<a class="faculty-link" href="/fsc/facultyProfile/1238"></a>
</div>
</div>


In [5]:
print('ID: ',int(LHRSinglefacultyMember.find('a',class_='faculty-link')['href'].split('/')[-1]))
print('Name:',str(LHRSinglefacultyMember.find('h5',class_='text-center').text))
print('Designation:',str(LHRSinglefacultyMember.find('p',class_='small text-center font-italic').text.split('\n')[0]))
print('HEC Approved?',bool('HEC Approved PhD Supervisor' in LHRSinglefacultyMember.find('p',class_='small text-center font-italic').text))
print('Email: ',str(LHRSinglefacultyMember.find('p',class_='mb-0 text-center').text))
print('Image URL:',urls[0].split('/f')[0]+LHRSinglefacultyMember.find('img',class_='card-img-top')['src'])

ID:  1238
Name: Dr. Kashif Zafar
Designation: Professor & HOD
HEC Approved? True
Email:  kashif.zafar@nu.edu.pk
Image URL: https://lhr.nu.edu.pk/media/Faculty/02_-_Dr._Kashif_Zafar_Prof__HoD_CS.png


In [6]:
# Scrapping Single Faculty Member's Profile Link
LHRSingleMemberProfileLink=urls[0].split('/f')[0]+LHRSinglefacultyMember.find('a',class_='faculty-link')['href']
print('Profile Link: ',LHRSingleMemberProfileLink)

Profile Link:  https://lhr.nu.edu.pk/fsc/facultyProfile/1238


In [7]:
# Making request to Single Faculty Member's Profile page to scrap further information i.e Highest Education, Extension & Department
requestLHRProfile=requests.get(LHRSingleMemberProfileLink)

if(requestLHRProfile.status_code==200):
    print(requestLHRProfile,'\nHTTP GET Request to Lahore Campus Faculty Member Profile Website Web Server Successful!\n')
    soupLHRSingleProfile=BeautifulSoup(requestLHRProfile.content,'html.parser') #Parsing Data Retrieved
    print(soupLHRSingleProfile.prettify()[0:100])
else:
    print(requestLHRProfile,'\nHTTP GET Request to Lahore Campus Faculty Member Profile Website Web Server Unsuccessful!\n')

<Response [200]> 
HTTP GET Request to Lahore Campus Faculty Member Profile Website Web Server Successful!

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-wid


In [8]:
print('Highest Education:',str(soupLHRSingleProfile.find(class_='col-lg-8 col-md-6 col-sm-12 text-justify')).split('<li>')[1].split('<')[0])
print('Extension: ',int(soupLHRSingleProfile.find('p',class_='mt-0 text-center').find('span').text.split(':')[1]))
print('Department:',str(soupLHRSingleProfile.find('a',class_='white ml-2 mr-2').text))

Highest Education:   Ph.D(Computer Science), NUCES, Islamabad, Pakistan (2010) 
Extension:  221
Department: FSC


In [10]:
# Now scraping info for all faculty members of Lahore Campus [General Code]
LHRfacultyMembersAll=soupLHR.findAll('div',class_='facultyCard')

Id=[]
name=[]
designation=[]
hecApprovedPhdSupervisor=[]
highestEducation=[]
email=[]
department=[]
extension=[]
profileLink=[]
imageUrl=[]

i=0

for facultyMember in LHRfacultyMembersAll:    
 
    name.append(str(facultyMember.find('h5',class_='text-center').text))
    designation.append(str(facultyMember.find('p',class_='small text-center font-italic').text.split('\n')[0]))
    hecApprovedPhdSupervisor.append(bool('HEC Approved PhD Supervisor' in facultyMember.find('p',class_='small text-center font-italic').text))
    email.append(str(facultyMember.find('p',class_='mb-0 text-center').text))
    imageUrl.append(urls[0].split('/f')[0]+facultyMember.find('img',class_='card-img-top')['src'])
    
    # Scrapping Faculty Member's Profile Link
    profileLink.append(urls[0].split('/f')[0]+facultyMember.find('a',class_='faculty-link')['href'])

    # Making request to Faculty Member's Profile page to scrap further information i.e Highest Education, Extension & Department    
    request2=requests.get(profileLink[i])
    if(request2.status_code!=200):
        print(request2,'Request to Faculty Member Profile Unsuccessful!',profileLink[i])
        break
        
    soupProfile=BeautifulSoup(request2.content,'html.parser')
    soupProfile.prettify()[0:1000]
    try: #Incase Education Details Not Available
        highestEducation.append(str(soupProfile.find(class_='col-lg-8 col-md-6 col-sm-12 text-justify')).split('<li>')[1].split('<')[0])
    except:
        highestEducation.append('Not Available')
        
    try: #Incase Faculty Member Does not have an extension
        extension.append(int(soupProfile.find('p',class_='mt-0 text-center').find('span').text.split(':')[1]))
    except:
        extension.append(-1)
    
    department.append(str(soupProfile.find('a',class_='white ml-2 mr-2').text))

    try: #Incase No int profile ID Available 
        Id.append(int(facultyMember.find('a',class_='faculty-link')['href'].split('/')[-1]))
    except:
        Id.append(-1)  
        print('No ID Found of',name[i]) 
        
    if((i+1)%10==0):#Tells number of Profiles Extracted after Every 10 Profiles Extracted
        print('Extracted information from',i+1,'Faculty Members profiles!')
        time.sleep(2)
    i=i+1
    
print(i,'Faculty Members Found!')

Extracted information from 10 Faculty Members profiles!
Extracted information from 20 Faculty Members profiles!
Extracted information from 30 Faculty Members profiles!
Extracted information from 40 Faculty Members profiles!
Extracted information from 50 Faculty Members profiles!
Extracted information from 60 Faculty Members profiles!
Extracted information from 70 Faculty Members profiles!
Extracted information from 80 Faculty Members profiles!
Extracted information from 90 Faculty Members profiles!
Extracted information from 100 Faculty Members profiles!
Extracted information from 110 Faculty Members profiles!
Extracted information from 120 Faculty Members profiles!
Extracted information from 130 Faculty Members profiles!
Extracted information from 140 Faculty Members profiles!
Extracted information from 150 Faculty Members profiles!
Extracted information from 160 Faculty Members profiles!
Extracted information from 170 Faculty Members profiles!
Extracted information from 180 Faculty M

In [11]:
facultyLahoreDF=pd.DataFrame({'ID':Id,'Name':name,'Designation':designation,
                            'HEC Approved PhD Supervisor':hecApprovedPhdSupervisor,
                            'Highest Education':highestEducation,
                            'Email':email,
                            'Deaprtment':department,'Extension':extension,'Profile Link':profileLink,'Image URL':imageUrl})

In [12]:
facultyLahoreDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 184 entries, 0 to 183
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ID                           184 non-null    int64 
 1   Name                         184 non-null    object
 2   Designation                  184 non-null    object
 3   HEC Approved PhD Supervisor  184 non-null    bool  
 4   Highest Education            184 non-null    object
 5   Email                        184 non-null    object
 6   Deaprtment                   184 non-null    object
 7   Extension                    184 non-null    int64 
 8   Profile Link                 184 non-null    object
 9   Image URL                    184 non-null    object
dtypes: bool(1), int64(2), object(7)
memory usage: 13.2+ KB


In [13]:
facultyLahoreDF.sample(5)

,ID,Name,Designation,HEC Approved PhD Supervisor,Highest Education,Email,Deaprtment,Extension,Profile Link,Image URL
136,9495,Dr. Zia Khan,Assistant Professor,True,"Ph.D(Management Sciences), Asian Institute o...",zia.khan@nu.edu.pk,FSM,252,https://lhr.nu.edu.pk/fsm/facultyProfile/9495,https://lhr.nu.edu.pk/media/Faculty/117-_Dr._Z...
99,5231,Mr. Syed Muhammad Ismail,Lecturer,False,"M.Sc(Power Electric Machines &amp; Drives), ...",muhammad.ismail@nu.edu.pk,EE,-1,https://lhr.nu.edu.pk/ee/facultyProfile/5231,https://lhr.nu.edu.pk/media/Faculty/110-_Syed_...
25,3720,Mr. Aamir Raheem,Assistant Professor,False,"M.S(Computer Science), NUCES,Lahore, Pakista...",aamir.raheem@nu.edu.pk,FSC,257,https://lhr.nu.edu.pk/fsc/facultyProfile/3720,https://lhr.nu.edu.pk/media/Faculty/18-_Aamir_...
49,6277,Ms. Maham Naeem,Lecturer,False,"MS (CS), NUCES, Lahore (2020)",maham.naeem@nu.edu.pk,FSC,-1,https://lhr.nu.edu.pk/fsc/facultyProfile/6277,https://lhr.nu.edu.pk/media/Faculty/6277.jpg
53,6415,Mr. Mubashar Hussain,Lecturer,False,"MS(CS), COMSATS, Lahore(2021)",mubashar.hussain@nu.edu.pk,FSC,-1,https://lhr.nu.edu.pk/fsc/facultyProfile/6415,https://lhr.nu.edu.pk/media/Faculty/30_-_Mubas...


In [14]:
facultyLahoreDF.to_csv('lhr.csv',index=False)

# ~ Islamabad Campus ~

In [15]:
driver=webdriver.Chrome()
driver.get(urls[1])
time.sleep(10) #Wait for whole content to load
ISLHTML=driver.page_source
driver.quit()
soupISB=BeautifulSoup(ISLHTML,'html.parser')
print(soupISB.prettify()[0:100])

<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compat


In [16]:
# First Writing Code to scrap details of a single Faculty Member of Islamabad Campus
ISLSinglefacultyMember=soupISB.find('div',class_='our-team')
print(ISLSinglefacultyMember)

<div class="our-team"><div class="pic"><img id="myImage4553" src="/Images/Profile/CS/4553.jpg"/></div><div class="team-content"><h3 class="title">Waseem Shahzad</h3><span class="post">Director &amp; Professor</span> <p>waseem.shahzad@nu.edu.pk</p><br/></div><ul class="social"><li><a href="/Faculty/Details/4553" style="font-style: italic;" target="_blank">More...</a></li></ul></div>


In [17]:
print('ID: ',int(ISLSinglefacultyMember.find('img')['src'].split('/')[-1].split('.')[0]))
print('Name:',str(ISLSinglefacultyMember.find('h3',class_='title').text))
print('Designation:',str(ISLSinglefacultyMember.find('span',class_='post').text))
print('Email: ',str(ISLSinglefacultyMember.find('p').text))
print('Image URL:',urls[1].split('/F')[0]+ISLSinglefacultyMember.find('img')['src'])
print('Profile Link: ',urls[1].split('/F')[0]+ISLSinglefacultyMember.find('a')['href'])

ID:  4553
Name: Waseem Shahzad
Designation: Director & Professor
Email:  waseem.shahzad@nu.edu.pk
Image URL: http://isb.nu.edu.pk/Images/Profile/CS/4553.jpg
Profile Link:  http://isb.nu.edu.pk/Faculty/Details/4553


In [18]:
# Making request to Faculty Member's Profile page to scrap further information i.e Highest Education, Extension & Department
requestISLProfile=urls[1].split('/F')[0]+ISLSinglefacultyMember.find('a')['href']
driver=webdriver.Chrome()
driver.get(requestISLProfile)
time.sleep(2) #Wait for whole content to load
ISLProfileSingleHtml=driver.page_source
driver.quit()

soupISLProfileSingle=BeautifulSoup(ISLProfileSingleHtml,'html.parser')
print(soupISLProfileSingle.prettify()[0:100])

<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compat


In [19]:
print('HEC Approved?',bool('HEC Approved PhD Supervisor' in soupISLProfileSingle.find('label',style='font-size:9pt;').text))
print('Extension: ',int(''.join(map(str,soupISLProfileSingle.findAll('p',style='line-height: 20px;margin-bottom: 0px;'))).split('class="fa fa-phone">')[1].split(': ')[1].split('</span')[0]))
print('Department:',str(soupISLProfileSingle.find('li',class_='breadcrumb-item active').text.strip()))
print('Highest Education:',str(soupISLProfileSingle.findAll('li',style='list-style-type:none;')[0].find('label').text))

HEC Approved? True
Extension:  101
Department: Computer Science
Highest Education: Ph.D (Computer Science) NUCES, (2010) 


In [20]:
# Now scraping info for all faculty members of Islamabad Campus [General Code]

Id=[]
name=[]
designation=[]
email=[]
profileLink=[]
imageUrl=[]

i=0
facultiesAllISL=soupISB.findAll('div',class_='our-team')

for facultyMember in facultiesAllISL:    
    try:
        Id.append(int(facultyMember.find('img')['src'].split('/')[-1].split('.')[0]))
    except:
        Id.append(-1)
    name.append(str(facultyMember.find('h3',class_='title').text))
    designation.append(str(facultyMember.find('span',class_='post').text))
    email.append(str(facultyMember.find('p').text))
    imageUrl.append(urls[1].split('/F')[0]+facultyMember.find('img')['src'])
    profileLink.append(urls[1].split('/F')[0]+facultyMember.find('a')['href'])
    
    i=i+1
    
print(i,'Faculty Members BASIC Information Found!')

204 Faculty Members BASIC Information Found!


In [21]:
# Scrapping Additional Information from Faculty Member's Profile Page
extension=[]
department=[]
hecApprovedPhdSupervisor=[]
highestEducation=[]
i=0
driver=webdriver.Chrome()

for profLinks in profileLink:
    # Making request to Faculty Member's Profile page to scrap further information i.e Highest Education, Extension & Department
    driver.get(profLinks)
    time.sleep(2) #Wait for whole page to load
    requ2=driver.page_source
    soup2B=BeautifulSoup(requ2,'html.parser')
    
    try:
        hecApprovedPhdSupervisor.append(bool('HEC Approved PhD Supervisor' in soup2B.find('label',style='font-size:9pt;').text))
    except:
        hecApprovedPhdSupervisor.append(False)
        
    try:
        extension.append(int(''.join(map(str,soup2B.findAll('p',style='line-height: 20px;margin-bottom: 0px;'))).split('class="fa fa-phone">')[1].split(': ')[1].split('</span')[0]))
    except:
        extension.append(-1)
        
    try:
        highestEducation.append(str(soup2B.findAll('li',style='list-style-type:none;')[0].find('label').text))
    except:
        highestEducation.append('Not Available')
        
    try:
        department.append(str(soup2B.find('li',class_='breadcrumb-item active').text.strip()))
    except:
        department.append('Not Available')
     
    i=i+1 
    if(i%10==0): #Tells How Much Scrapped after every 10 profiles visited
        print('Extracted Information from',i,'Faculty Member Profiles!')
        
driver.quit()
print(i,'Faculty Members FULL Information Found!')

Extracted Information from 10 Faculty Member Profiles!
Extracted Information from 20 Faculty Member Profiles!
Extracted Information from 30 Faculty Member Profiles!
Extracted Information from 40 Faculty Member Profiles!
Extracted Information from 50 Faculty Member Profiles!
Extracted Information from 60 Faculty Member Profiles!
Extracted Information from 70 Faculty Member Profiles!
Extracted Information from 80 Faculty Member Profiles!
Extracted Information from 90 Faculty Member Profiles!
Extracted Information from 100 Faculty Member Profiles!
Extracted Information from 110 Faculty Member Profiles!
Extracted Information from 120 Faculty Member Profiles!
Extracted Information from 130 Faculty Member Profiles!
Extracted Information from 140 Faculty Member Profiles!
Extracted Information from 150 Faculty Member Profiles!
Extracted Information from 160 Faculty Member Profiles!
Extracted Information from 170 Faculty Member Profiles!
Extracted Information from 180 Faculty Member Profiles!
E

In [22]:
facultyIslamabadDF=pd.DataFrame({'ID':Id,'Name':name,'Designation':designation,
                            'HEC Approved PhD Supervisor':hecApprovedPhdSupervisor,
                            'Highest Education':highestEducation,
                            'Email':email,
                            'Deaprtment':department,'Extension':extension,
                            'Profile Link':profileLink,'Image URL':imageUrl})

In [23]:
facultyIslamabadDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204 entries, 0 to 203
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ID                           204 non-null    int64 
 1   Name                         204 non-null    object
 2   Designation                  204 non-null    object
 3   HEC Approved PhD Supervisor  204 non-null    bool  
 4   Highest Education            204 non-null    object
 5   Email                        204 non-null    object
 6   Deaprtment                   204 non-null    object
 7   Extension                    204 non-null    int64 
 8   Profile Link                 204 non-null    object
 9   Image URL                    204 non-null    object
dtypes: bool(1), int64(2), object(7)
memory usage: 14.7+ KB


In [24]:
facultyIslamabadDF.sample(5)

,ID,Name,Designation,HEC Approved PhD Supervisor,Highest Education,Email,Deaprtment,Extension,Profile Link,Image URL
165,9151,Usman Zafar Chaudhry,Assistant Professor,False,MSc (International Business & Management) Bra...,usman.chaudhry@nu.edu.pk,FAST School of Management,347,http://isb.nu.edu.pk/Faculty/Details/9151,http://isb.nu.edu.pk/Images/Profile/FSM/9151.JPG
202,9341,Sadia Nauman,Lecturer,False,M.Phil (English) Government College Universit...,sadia.nauman@nu.edu.pk,Science & Humanities,532,http://isb.nu.edu.pk/Faculty/Details/9341,http://isb.nu.edu.pk/Images/Profile/SH/9341.jpg
98,6406,Sher Bano\n,Instructor,False,BS (Computer Science) National University of C...,sher.bano@nu.edu.pk \n,Computer Science,-1,http://isb.nu.edu.pk/Faculty/Details/6406,http://isb.nu.edu.pk/Images/Profile/CS/6406.jpg
59,6247,Pir Sami Ullah Shah,Lecturer,False,Not Available,samiullah.shah@nu.edu.pk,Computer Science,321,http://isb.nu.edu.pk/Faculty/Details/6247,http://isb.nu.edu.pk/Images/Profile/CS/6247.jpg
22,4708,Uzair Khan,Associate Professor,True,"P.hD (Computer Science) Inria-i3s CNRS UNSA, F...",uzair.khan@nu.edu.pk,Computer Science,505,http://isb.nu.edu.pk/Faculty/Details/4708,http://isb.nu.edu.pk/Images/Profile/CS/4708.jpg


In [25]:
facultyIslamabadDF.to_csv('isl.csv',index=False)

# ~ Karachi Campus ~

In [26]:
requestKHI=requests.get(urls[2])
if(requestKHI.status_code==200):
    print(requestKHI,'\nHTTP GET Request to Karachi Campus Website Web Server Successful!\n')
    soupKHI=BeautifulSoup(requestKHI.content,'html.parser') #Parsing Data Retrieved
    print(soupKHI.prettify()[0:100])
else:
    print(requestKHI,'\nHTTP GET Request to Karachi Campus Website Web Server Unsuccessful!\n')

<Response [200]> 
HTTP GET Request to Karachi Campus Website Web Server Successful!

<!DOCTYPE html>
<html class="no-js" lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content=


In [27]:
# Gathering All department Links
deptLinks=soupKHI.findAll('a')
departmentLINKS=[]
i=0
for dept in deptLinks:
    if(i>=7):
        break
    if('Department' in dept.text):
        print(dept['href'])
        departmentLINKS.append(dept['href'])
        i=i+1

https://khi.nu.edu.pk/faculty-php/
https://khi.nu.edu.pk/department-of-cyber-security/
https://khi.nu.edu.pk/department-of-artificial-intelligence/
https://khi.nu.edu.pk/department-of-software-engineering/
https://khi.nu.edu.pk/department-of-electrical-engineering/
https://khi.nu.edu.pk/department-of-management-sciences/
https://khi.nu.edu.pk/department-of-sciences-humanities/


In [28]:
facultiesKarachiProfileLinks={} #Gathering Profile links of all departments all Faculty Members
for i in range(len(departmentLINKS)):
    requestKHIA=requests.get(departmentLINKS[i])
    soupKHIA=BeautifulSoup(requestKHIA.content,'html.parser')
    facultyMemKHI=soupKHIA.findAll('div',class_='gdlr-core-personnel-list-image gdlr-core-media-image gdlr-core-hover-element gdlr-core-zoom-on-hover')
    facultyLINKS=[]
    for j in range(len(facultyMemKHI)):
        facultyLINKS.append(facultyMemKHI[j].find('a')['href'])
    
    if(i==0):
        facultiesKarachiProfileLinks['department-of-computer-science']=facultyLINKS
    else:
        facultiesKarachiProfileLinks[departmentLINKS[i].split('/')[-2]]=facultyLINKS
    
print(facultiesKarachiProfileLinks)

{'department-of-computer-science': ['https://khi.nu.edu.pk/personnel/prof-dr-zulfiqar-ali-memon-phd/', 'https://khi.nu.edu.pk/personnel/dr-farooque-hassan-kumbhar-phd/', 'https://khi.nu.edu.pk/personnel/dr-jawwad-a-shamsi-phd/', 'https://khi.nu.edu.pk/personnel/dr-muhammad-atif-tahir-phd/', 'https://khi.nu.edu.pk/personnel/dr-ghufran-ahmed-phd/', 'https://khi.nu.edu.pk/personnel/dr-nadeem-kafi-phd/', 'https://khi.nu.edu.pk/personnel/dr-nouman-durrani-phd/', 'https://khi.nu.edu.pk/personnel/dr-fahad-sherwani-phd/', 'https://khi.nu.edu.pk/personnel/dr-farrukh-hasan-syed-phd/', 'https://khi.nu.edu.pk/personnel/dr-farrukh-salim-shaikh-phd/', 'https://khi.nu.edu.pk/personnel/dr-muhammad-farrukh-shahid-phd/', 'https://khi.nu.edu.pk/personnel/dr-nausheen-shoaib-phd/', 'https://khi.nu.edu.pk/personnel/mr-muhammad-jamil/', 'https://khi.nu.edu.pk/personnel/mr-muhammad-shahzad/', 'https://khi.nu.edu.pk/personnel/mr-shahbaz-siddiqui/', 'https://khi.nu.edu.pk/personnel/dr-nausheen-shoaib/', 'https:

In [29]:
# First Writing Code to scrap details of a single Faculty Member
requestKHIFacultyProfileSingle=requests.get(facultiesKarachiProfileLinks['department-of-computer-science'][0])
soupKHIProfile=BeautifulSoup(requestKHIFacultyProfileSingle.content,'html.parser')
print(soupKHIProfile.prettify()[0:100])

<!DOCTYPE html>
<html class="no-js" lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content=


In [30]:
print('ID: ',int(str(soupKHIProfile.find(id='burst-js-extra')).split('\"page_id\":')[1].split("\"")[1]))
print('Profile Link:',facultiesKarachiProfileLinks['department-of-computer-science'][0])
print('Name:',str(soupKHIProfile.find('h3',class_='gdlr-core-title-item-title gdlr-core-skin-title').text.split(',')[0]))
print('Designation:',str(soupKHIProfile.find('span',class_='gdlr-core-title-item-caption gdlr-core-info-font gdlr-core-skin-caption').text.split('\n')[0]))
print('HEC Approved?',bool('HEC Approved PhD Supervisor' in soupKHIProfile.find('span',class_='gdlr-core-title-item-caption gdlr-core-info-font gdlr-core-skin-caption').text))
print('Email: ',str(soupKHIProfile.find('div',class_='kingster-personnel-info-list kingster-type-email').text))
print('Image URL:',soupKHIProfile(class_='gdlr-core-lightgallery gdlr-core-js')[0]['href'])
print('Highest Education:',str(''.join(map(str,soupKHIProfile.findAll('div',class_='gdlr-core-pbf-element'))).split('Education')[1].split('</span')[1].split('>')[-1]))
print('Extension: ',int(soupKHIProfile.find('div',class_='kingster-personnel-info-list kingster-type-phone').text))
print('Department:',soupKHIProfile(class_='gdlr-core-lightgallery gdlr-core-js')[0]['href'].split('-')[-1].split('.')[0])

ID:  6281
Profile Link: https://khi.nu.edu.pk/personnel/prof-dr-zulfiqar-ali-memon-phd/
Name: Prof. Dr. Zulfiqar Ali Memon 
Designation: Professor and HoS
HEC Approved? True
Email:  zulfiqar.memon@nu.edu.pk
Image URL: https://khi.nu.edu.pk/wp-content/uploads/2023/01/Dr-Zulfiqar-Ali-Memon-HOS-FSC.png
Highest Education: PhD (Vrije University) Amsterdam, The Netherlands
Extension:  214
Department: FSC


In [32]:
# Now scraping info for all faculty members of Karachi Campus [General Code]

Id=[]
name=[]
designation=[]
hecApprovedPhdSupervisor=[]
highestEducation=[]
email=[]
department=[]
extension=[]
imageUrl=[]
profiles=[]

for departmentName,profileLinks in facultiesKarachiProfileLinks.items():
    for i in range(len(profileLinks)):   
        requestKHIFacultyProfiles=requests.get(profileLinks[i])
        profiles.append(profileLinks[i])
        soupKHIProfiles=BeautifulSoup(requestKHIFacultyProfiles.content,'html.parser')
        Id.append(int(str(soupKHIProfiles.find(id='burst-js-extra')).split('\"page_id\":')[1].split("\"")[1]))
        name.append(str(soupKHIProfiles.find('h3',class_='gdlr-core-title-item-title gdlr-core-skin-title').text.split(',')[0]))
        designation.append(str(soupKHIProfiles.find('span',class_='gdlr-core-title-item-caption gdlr-core-info-font gdlr-core-skin-caption').text.split('\n')[0]))
        hecApprovedPhdSupervisor.append(bool('HEC Approved PhD Supervisor' in soupKHIProfiles.find('span',class_='gdlr-core-title-item-caption gdlr-core-info-font gdlr-core-skin-caption').text))
        
        try: #Incase Email Not Available
            email.append(str(soupKHIProfiles.find('div',class_='kingster-personnel-info-list kingster-type-email').text))
        except:
            email.append('Not Available')
            
        imageUrl.append(soupKHIProfiles(class_='gdlr-core-lightgallery gdlr-core-js')[0]['href'])
        
        try: #Incase Education Details Not Available
            highestEducation.append(str(''.join(map(str,soupKHIProfiles.findAll('div',class_='gdlr-core-pbf-element'))).split('Education')[1].split('</span')[1].split('>')[-1]))
            
        except:
            highestEducation.append('Not Available')
            
        try: #Incase Extension Not Available
            extension.append(int(soupKHIProfiles.find('div',class_='kingster-personnel-info-list kingster-type-phone').text))
        except:
            extension.append(-1)
        
        department.append(departmentName)

        if(len(profiles)%10==0): #Tells how many faculty members profile information extracted after every 10 profiles extracted
            print('Extracted information from',len(profiles),'Faculty Members Profiles!')
    
print(len(profiles),'Faculty Members Found!')

Extracted information from 10 Faculty Members Profiles!
Extracted information from 20 Faculty Members Profiles!
Extracted information from 30 Faculty Members Profiles!
Extracted information from 40 Faculty Members Profiles!
Extracted information from 50 Faculty Members Profiles!
Extracted information from 60 Faculty Members Profiles!
Extracted information from 70 Faculty Members Profiles!
Extracted information from 80 Faculty Members Profiles!
Extracted information from 90 Faculty Members Profiles!
Extracted information from 100 Faculty Members Profiles!
Extracted information from 110 Faculty Members Profiles!
Extracted information from 120 Faculty Members Profiles!
Extracted information from 130 Faculty Members Profiles!
132 Faculty Members Found!


In [33]:
facultyKarachiDF=pd.DataFrame({'ID':Id,'Name':name,'Designation':designation,
                            'HEC Approved PhD Supervisor':hecApprovedPhdSupervisor,
                            'Highest Education':highestEducation,
                            'Email':email,
                            'Deaprtment':department,'Extension':extension,'Profile Link':profiles,'Image URL':imageUrl})

In [34]:
facultyKarachiDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 132 entries, 0 to 131
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ID                           132 non-null    int64 
 1   Name                         132 non-null    object
 2   Designation                  132 non-null    object
 3   HEC Approved PhD Supervisor  132 non-null    bool  
 4   Highest Education            132 non-null    object
 5   Email                        132 non-null    object
 6   Deaprtment                   132 non-null    object
 7   Extension                    132 non-null    int64 
 8   Profile Link                 132 non-null    object
 9   Image URL                    132 non-null    object
dtypes: bool(1), int64(2), object(7)
memory usage: 9.5+ KB


In [35]:
facultyKarachiDF.sample(5)

,ID,Name,Designation,HEC Approved PhD Supervisor,Highest Education,Email,Deaprtment,Extension,Profile Link,Image URL
62,6602,Syed Farooq Zaidi,Senior Lecturer,False,"MS (CS), University of Massachusetts, USA (2012)",Not Available,department-of-artificial-intelligence,-1,https://khi.nu.edu.pk/personnel/syed-farooq-za...,https://khi.nu.edu.pk/wp-content/uploads/2023/...
37,7293,Mr. Shahroz Bakht,Lecturer,False,"MS (CS), NUCES, Karachi",shahroz.bakht@nu.edu.pk,department-of-computer-science,136,https://khi.nu.edu.pk/personnel/mr-shahroz-bakht/,https://khi.nu.edu.pk/wp-content/uploads/2023/...
76,6981,Dr. Burhan Khan,Assistant Professor & HoD,False,"PhD (Engg.), University of Tunku Abdul Rahman,...",burhan.khan@nu.edu.pk,department-of-electrical-engineering,265,https://khi.nu.edu.pk/personnel/dr-burhan-khan...,https://khi.nu.edu.pk/wp-content/uploads/2023/...
115,7480,Ms. Javeria Iftikhar,Lecturer,False,"MS (Math), IoBM, Karachi",javeria.iftikhar@nu.edu.pk,department-of-sciences-humanities,112,https://khi.nu.edu.pk/personnel/ms-javeria-ift...,https://khi.nu.edu.pk/wp-content/uploads/2023/...
46,7369,Mr. Shaheer Ahmad Khan,Instructor,False,"BS (IT), SS-UET, Karachi",shaheer.khan@nu.edu.pk,department-of-computer-science,284,https://khi.nu.edu.pk/personnel/mr-shaheer-ahm...,https://khi.nu.edu.pk/wp-content/uploads/2023/...


In [36]:
facultyKarachiDF.to_csv('khi.csv',index=False)

## Step 2:
### Concatenate all data frames to create a single data frame having the information of all faculty members from all campuses. Save this data frame as a .csv file named fast_faculty.csv.


In [37]:
lhr=pd.read_csv('lhr.csv')
isl=pd.read_csv('isl.csv')
khi=pd.read_csv('khi.csv')

In [39]:
fast_faculty=pd.concat([lhr,isl,khi])
fast_faculty.sample(5)

,ID,Name,Designation,HEC Approved PhD Supervisor,Highest Education,Email,Deaprtment,Extension,Profile Link,Image URL
56,6694,Ms. Rabia Ahmed Ansari,Lecturer,False,"MS (CNS), NUCES, Karachi",rabia.ansari@nu.edu.pk,department-of-cyber-security,293,https://khi.nu.edu.pk/personnel/ms-rabia-ahmed...,https://khi.nu.edu.pk/wp-content/uploads/2023/...
12,4335,Dr. Asma Ahmad,Assistant Professor,False,"PhD(CS), University of Connecticut",asma.ahmad@nu.edu.pk,FSC,-1,https://lhr.nu.edu.pk/fsc/facultyProfile/4335,https://lhr.nu.edu.pk/media/Faculty/47_-_Dr._A...
88,7032,Ms. Rukhsar Ali,Lecturer,False,"MS (EE), NUCES, Karachi",rukhsar.ali@nu.du.pk,department-of-electrical-engineering,281,https://khi.nu.edu.pk/personnel/ms-rukhsar-ali/,https://khi.nu.edu.pk/wp-content/uploads/2023/...
129,4340,Sana Hassan,Assistant Professor,False,MS (Electrical Enigeering) National University...,sana.hassan@nu.edu.pk,Electrical Engineering,105,http://isb.nu.edu.pk/Faculty/Details/4340,http://isb.nu.edu.pk/Images/Profile/EE/4340.jpg
72,6663,Ms. Zoha Waheed,Instructor,False,"BS, NUCES, Lahore (2018)",zoha.waheed@nu.edu.pk,FSC,-1,https://lhr.nu.edu.pk/fsc/facultyProfile/6663,https://lhr.nu.edu.pk/media/Faculty/6663.png


In [40]:
fast_faculty.to_csv('fast_faculty.csv',index=False)

## Step 3:
### Load fast_faculty.csv into a data frame named faculty. Divide last digit of your Student ID by 10 and pass it into sample method as frac value. If the last digit of your Student ID is 0, keep the below code as -it-is. Save the sample_ data frame as sample.csv file. sample_ = faculty.sample(frac=0.1)


In [41]:
faculty=pd.read_csv('fast_faculty.csv')

In [42]:
sample_=faculty.sample(frac=0.1) #Roll Num : 0921 , 1/10=0.1

In [43]:
sample_.to_csv('sample.csv',index=False)

# ~ Bonus ~

In [44]:
# BONUS:  I wrote this part to futher explore the data I extracted
import pandas as pd
from IPython.display import display
def searchFacultyMembersFAST():
    '''
    This function lets user search a faculty member in Fast Lahore,Islamabad & Karachi Campus.
    -Input: Takes name of faculty member as input from user 
    -Output : Prints their Relevant Information like:
                      -> ID 
                      -> Name
                      -> Designation
                      -> HEC Approced PHD supervisor or not
                      -> Highest Education
                      -> Email
                      -> Department
                      -> Extension 
                      -> ImageURL
    '''
    while(True):
        name=input('\nPlease Enter Name of Faculty Member to Search or exit to Quit: ')
        
        if(name.lower()=='exit' or name.lower()=='quit'):
            print('BYE!')
            break
        
        #To make sure names entered in both lowercase and uppercase are matched
        fastFaculty=pd.read_csv('fast_faculty.csv')
        fast_faculty2=fastFaculty.copy()
        fast_faculty2['Name']=fastFaculty['Name'].str.lower()
        
        results=fastFaculty[fast_faculty2.Name.str.contains(name.lower())]
        
        if(len(results)>0):
            print('\nYour Search Results Are as Follows:\n')
            display(results)
        else:
            print('\nNo match found!\n')
        
help(searchFacultyMembersFAST)
searchFacultyMembersFAST()

Help on function searchFacultyMembersFAST in module __main__:

searchFacultyMembersFAST()
    This function lets user search a faculty member in Fast Lahore,Islamabad & Karachi Campus.
    -Input: Takes name of faculty member as input from user 
    -Output : Prints their Relevant Information like:
                      -> ID 
                      -> Name
                      -> Designation
                      -> HEC Approced PHD supervisor or not
                      -> Highest Education
                      -> Email
                      -> Department
                      -> Extension 
                      -> ImageURL


Please Enter Name of Faculty Member to Search or exit to Quit: Aisha

Your Search Results Are as Follows:



,ID,Name,Designation,HEC Approved PhD Supervisor,Highest Education,Email,Deaprtment,Extension,Profile Link,Image URL
168,5622,Ms. Aisha Bano,Lecturer,False,"MPhil,hazara university,MANSEHRA(2016)",aisha.bano@nu.edu.pk,SS,-1,https://lhr.nu.edu.pk/ss/facultyProfile/5622,https://lhr.nu.edu.pk/media/Faculty/164_-_Aish...
380,3940,Aisha Ijaz,Assistant Professor,False,"M.Phil (Physics) Quaid-i-Azam University , Isl...",aisha.ijaz@nu.edu.pk,Science & Humanities,362,http://isb.nu.edu.pk/Faculty/Details/3940,http://isb.nu.edu.pk/Images/Profile/SH/3940.jpg



Please Enter Name of Faculty Member to Search or exit to Quit: Mehwish

Your Search Results Are as Follows:



,ID,Name,Designation,HEC Approved PhD Supervisor,Highest Education,Email,Deaprtment,Extension,Profile Link,Image URL
381,4127,Mehwish Hassan,Assistant Professor,False,"P.hD (Maths, Engineering) University of Bresci...",mehwish.hassan@nu.edu.pk,Science & Humanities,363,http://isb.nu.edu.pk/Faculty/Details/4127,http://isb.nu.edu.pk/Images/Profile/SH/4127.jpg



Please Enter Name of Faculty Member to Search or exit to Quit: ishaq

Your Search Results Are as Follows:



,ID,Name,Designation,HEC Approved PhD Supervisor,Highest Education,Email,Deaprtment,Extension,Profile Link,Image URL
27,4015,Mr. Ishaq Raza,Assistant Professor,False,"M.S(Computer Science), University Of Central...",ishaq.raza@nu.edu.pk,FSC,214,https://lhr.nu.edu.pk/fsc/facultyProfile/4015,https://lhr.nu.edu.pk/media/Faculty/17_-_Ishaq...



Please Enter Name of Faculty Member to Search or exit to Quit: saif

Your Search Results Are as Follows:



,ID,Name,Designation,HEC Approved PhD Supervisor,Highest Education,Email,Deaprtment,Extension,Profile Link,Image URL
48,6588,Mr. M. Saif Ul Islam,Lecturer,False,"MS (DS), NUCES, Karachi (2019)",muhammad.saif@nu.edu.pk,FSC,520,https://lhr.nu.edu.pk/fsc/facultyProfile/6588,https://lhr.nu.edu.pk/media/Faculty/31-_M._Sai...
59,6268,Mr. Saifullah Tanvir,Lecturer,False,"BS (CS), Forman Christian College, Lahore, 2...",saifullah.tanvir@nu.edu.pk,FSC,-1,https://lhr.nu.edu.pk/fsc/facultyProfile/6268,https://lhr.nu.edu.pk/media/Faculty/26_-_Saifu...
261,6609,Shahela Saif,Lecturer,False,MS (Computer Software Engineering) National Un...,shahela.saif@nu.edu.pk,Computer Science,347,http://isb.nu.edu.pk/Faculty/Details/6609,http://isb.nu.edu.pk/Images/Profile/CS/6609.jpg
323,-1,Hassan Saif,Assistant Professor,False,"PhD ( Enigneering) Sungkyunkwan University, Ko...",hassan.saif@nu.edu.pk,Electrical Engineering,-1,http://isb.nu.edu.pk/Faculty/Details/5953,http://isb.nu.edu.pk/Images/user/avatar_2x.png
362,5141,Saifullah Khan,Lecturer,False,"MS (Finance) International Islamic University,...",saifullah.khan@nu.edu.pk,FAST School of Management,166,http://isb.nu.edu.pk/Faculty/Details/5141,http://isb.nu.edu.pk/Images/Profile/FSM/5141.jpg



Please Enter Name of Faculty Member to Search or exit to Quit: quit
BYE!
